## Loading import

In [32]:
import numpy as np
import laspy

from point_cloud_segmentation.config import RAW_DATA_DIR
from point_cloud_segmentation.visualizing.visualise import *


## Loading data

In [33]:
# las = laspy.read(RAW_DATA_DIR / "wkd-house_1_8xds.laz")
# las = laspy.read(RAW_DATA_DIR / "wkd-house_2_8xds.laz")
las = laspy.read(RAW_DATA_DIR / "wkd-half_house_2_8xds.laz")
print(np.unique(las.classification))

points = np.vstack((las.x, las.y, las.z)).transpose()

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

print(pcd)


[0]
PointCloud with 156485426 points.


## Downsampling

In [34]:
voxel_size = 0.01
ds_pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
print(ds_pcd)

PointCloud with 17856528 points.


## DBSCAN Segmentation

In [35]:
# ds_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
ds_pcd_np = np.array(ds_pcd.points)
# ds_pcd_np = np.array(pcd.points)
# ds_pcd = pcd

print(ds_pcd)
print(ds_pcd_np.shape)


PointCloud with 17856528 points.
(17856528, 3)


In [36]:
# epsilon = voxel_size * 1.5
epsilon = voxel_size * 2
min_points = 10
labels = np.array(ds_pcd.cluster_dbscan(eps=epsilon, min_points=min_points, print_progress=True))

print(labels)
print(labels.shape)

inlier_mask = labels >= 0
outlier_mask = labels == -1

inliers = ds_pcd_np[inlier_mask]
outliers = ds_pcd_np[outlier_mask]
print(inliers.shape)
print(outliers.shape)

inliers_pcd = o3d.geometry.PointCloud()
outliers_pcd = o3d.geometry.PointCloud()
inliers_pcd.points = o3d.utility.Vector3dVector(inliers)
outliers_pcd.points = o3d.utility.Vector3dVector(outliers)
print(inliers_pcd)
print(outliers_pcd)

visualize_geometries([inliers_pcd, outliers_pcd])


[-1  0  0 ...  0  0  0]
(17856528,)
(16701791, 3)
(1154737, 3)
PointCloud with 16701791 points.
PointCloud with 1154737 points.


In [40]:
# Create a color array (black for noise)
max_label = labels.max()
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0  # Set noise (-1) to black
print(max_label)

# Apply colors to the original point cloud
ds_pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([ds_pcd])

21028
